## chap8. データラングリング：連結、結合、変形  
2023-11-5 ~ 11-6

## 8-1. 階層型インデックス

In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
data = pd.Series(np.random.uniform(size=9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.280385
   2    0.052456
   3    0.382045
b  1    0.722985
   3    0.960506
c  1    0.138263
   2    0.087119
d  2    0.942850
   3    0.135823
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data['b']

1    0.722985
3    0.960506
dtype: float64

In [5]:
data[['b', 'd']]

b  1    0.722985
   3    0.960506
d  2    0.942850
   3    0.135823
dtype: float64

In [6]:
data.loc[['b','d']]

b  1    0.722985
   3    0.960506
d  2    0.942850
   3    0.135823
dtype: float64

In [7]:
data.loc[:,2]

a    0.052456
c    0.087119
d    0.942850
dtype: float64

In [8]:
data.unstack()

,1,2,3
a,0.280385,0.052456,0.382045
b,0.722985,NaN,0.960506
c,0.138263,0.087119,NaN
d,NaN,0.942850,0.135823


In [9]:
data.unstack().stack()

a  1    0.280385
   2    0.052456
   3    0.382045
b  1    0.722985
   3    0.960506
c  1    0.138263
   2    0.087119
d  2    0.942850
   3    0.135823
dtype: float64

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                     columns=[['Ohio', 'Ohio','Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame.index.nlevels

2

In [13]:
frame.columns.nlevels

2

In [14]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

8.1.1 階層の順序変更やソート

In [15]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [16]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [17]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

8.1.2 階層ごとの要約統計量

In [18]:
frame.groupby(level='key2').sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [19]:
frame.groupby(level='color', axis='columns').sum()

C:\Users\sinis\AppData\Local\Temp\ipykernel_13028\3080437908.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level='color', axis='columns').sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

8.1.3 データフレームの列をインデックスに使う

In [20]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [21]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [22]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [23]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [24]:
frame2.sort_index()

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

### 8.2 データセットの結合とマージ

In [25]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'data1': pd.Series(range(7), dtype='Int64')})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 
                    'data2': pd.Series(range(3), dtype='Int64')})

In [26]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [27]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [28]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [29]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'data1': pd.Series(range(7), dtype='Int64')})
df4 = pd.DataFrame({'rkey':['a', 'b', 'd'], 
                    'data2': pd.Series(range(3), dtype='Int64')})

In [30]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [31]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [32]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [33]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [34]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [35]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': pd.Series(range(6), dtype='Int64')})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 
                    'data2': pd.Series(range(5), dtype='Int64')})

In [36]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [37]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [38]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [39]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 
                     'key2': ['one', 'two', 'one'], 
                     'lval': pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 
                      'key2': ['one', 'one', 'one', 'two'], 
                      'rval': pd.Series([4, 5, 6, 7], dtype='Int64')})

In [40]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [41]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [43]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


In [44]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [46]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


8.2.2 インデックスによるマージ

In [47]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 
                      'value': pd.Series(range(6), dtype='Int64')})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [48]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [49]:
right1

,group_val
a,3.5
b,7.0


In [50]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [51]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [52]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 
                      'key2': [2000, 2001, 2002, 2001, 2002], 
                      'data': pd.Series(range(5), dtype='Int64')})
righth_index = pd.MultiIndex.from_arrays(
    [
        ['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'], 
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

In [53]:
righth = pd.DataFrame({'event1': pd.Series([0, 2, 4, 6, 8, 10], dtype='Int64', index=righth_index),
                       'event2': pd.Series([1, 3, 5, 7, 9, 11], dtype='Int64', index=righth_index)})

In [54]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [55]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [58]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [59]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


In [60]:
left2 = pd.DataFrame([[1., 2.],[3., 4.], [5., 6.]], 
                      index=['a', 'c', 'e'], 
                      columns=['Ohio', 'Nevada']).astype('Int64')
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]], 
                      index=['b', 'c', 'd', 'e'], 
                      columns=['Missouri', 'Alabama']).astype('Int64')

In [61]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [62]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [65]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [66]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [67]:
another = pd.DataFrame([[7., 8.],[9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [70]:
left2.join([right2, another], how='outer', sort=True)

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
b,<NA>,<NA>,7,8,NaN,NaN
c,3,4,9,10,9.0,10.0
d,<NA>,<NA>,11,12,NaN,NaN
e,5,6,13,14,11.0,12.0
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


8.2.3 軸に沿った連結

In [71]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [73]:
np.concatenate([arr,arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [75]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [76]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [77]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [78]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [79]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [80]:
df1 = pd.DataFrame(np.arange(6).reshape((3, 2)), index=['a', 'b', 'c'], columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape((2, 2)), index=['a', 'c'], columns=['three', 'four'])

In [81]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [82]:
df2

,three,four
a,5,6
c,7,8


In [83]:
pd.concat([df1, df2], axis=1, keys=['level1','level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [84]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [85]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [86]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)), 
                   columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)), 
                   columns=['b', 'd', 'a'])

In [87]:
df1

,a,b,c,d
0,0.845492,0.219993,-0.489561,-1.565368
1,0.530972,-0.754611,1.245749,0.082702
2,0.170836,-1.263314,0.725882,-0.827652


In [88]:
df2

,b,d,a
0,-0.285483,1.118731,0.719474
1,0.920560,-0.127068,-0.692211


In [90]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.845492,0.219993,-0.489561,-1.565368
1,0.530972,-0.754611,1.245749,0.082702
2,0.170836,-1.263314,0.725882,-0.827652
3,0.719474,-0.285483,NaN,1.118731
4,-0.692211,0.920560,NaN,-0.127068


重複のあるデータの結合

In [92]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan], 
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=['a', 'b', 'c', 'd', 'e', 'f'])

In [93]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [94]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [95]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [96]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [97]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan], 
                    'b': [np.nan, 2., np.nan, 6.], 
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.], 
                    'b': [np.nan, 3., 4., 6., 8.]})

In [98]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [99]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [100]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 変形とピボット操作

stack ...データ内の各列を行へとピボット（回転）させる  
unstack ...各行を列へと回転させる

In [101]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(['Ohio', 'Colorado',], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))

In [102]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [104]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [105]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [106]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [107]:
result.unstack(level='state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [108]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [109]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [110]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [111]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [115]:
data2.unstack().stack(dropna=False).astype('Int64')

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [116]:
df = pd.DataFrame({'left': result, 'right': result + 5}, 
                  columns=pd.Index(['left', 'right'], name='side'))

In [117]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [118]:
df.unstack(level='state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [119]:
df.unstack(level='state').stack(level='side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

8.3.2 「縦持ち」形式から「横持ち」形式へのピボット

In [120]:
data = pd.read_csv('examples/macrodata.csv')

In [121]:
data = data.loc[:, ['year', 'quarter', 'realgdp', 'infl', 'unemp']]

In [122]:
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [123]:
periods = pd.PeriodIndex(year=data.pop('year'), 
                         quarter=data.pop('quarter'),
                         name='date')

In [124]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [125]:
data.index = periods.to_timestamp('D')

In [126]:
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [127]:
data.columns.name = 'item'

In [128]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [130]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0:'value'}))

In [133]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [134]:
pivoted = long_data.pivot(index='date', columns='item', values='value')

In [135]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [136]:
long_data['value2'] = np.random.standard_normal(len(long_data))

In [137]:
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,0.054122
1,1959-01-01,infl,0.000,-1.140402
2,1959-01-01,unemp,5.800,-0.798680
3,1959-04-01,realgdp,2778.801,-0.497611
4,1959-04-01,infl,2.340,-0.581970
5,1959-04-01,unemp,5.100,-0.255385
6,1959-07-01,realgdp,2775.488,-0.843328
7,1959-07-01,infl,2.740,0.371876
8,1959-07-01,unemp,5.300,-0.901965
9,1959-10-01,realgdp,2785.204,0.502126


In [138]:
pivoted = long_data.pivot(index='date', columns='item')

In [139]:
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -1.140402  0.054122 -0.798680
1959-04-01  2.34  2778.801   5.1 -0.581970 -0.497611 -0.255385
1959-07-01  2.74  2775.488   5.3  0.371876 -0.843328 -0.901965
1959-10-01  0.27  2785.204   5.6 -1.212185  0.502126  2.278724
1960-01-01  2.31  2847.699   5.2 -0.710704 -0.709024  0.907881

In [141]:
unstacked = long_data.set_index(['date', 'item']).unstack(level='item')

In [142]:
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -1.140402  0.054122 -0.798680
1959-04-01  2.34  2778.801   5.1 -0.581970 -0.497611 -0.255385
1959-07-01  2.74  2775.488   5.3  0.371876 -0.843328 -0.901965
1959-10-01  0.27  2785.204   5.6 -1.212185  0.502126  2.278724
1960-01-01  2.31  2847.699   5.2 -0.710704 -0.709024  0.907881

8.3.3 「横持ち」形式から「縦持ち」形式へのピボット

In [146]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'B': [4, 5, 6],
                   'A': [1, 2, 3], 
                   'C': [7, 8, 9]})

In [147]:
df

,key,B,A,C
0,foo,4,1,7
1,bar,5,2,8
2,baz,6,3,9


In [148]:
melted = pd.melt(df, id_vars='key')

In [149]:
melted

,key,variable,value
0,foo,B,4
1,bar,B,5
2,baz,B,6
3,foo,A,1
4,bar,A,2
5,baz,A,3
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [152]:
reshaped = melted.pivot(index='key', columns='variable', values='value')

In [153]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [154]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [155]:
pd.melt(df, id_vars='key', value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [156]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [157]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
